# ENGR-371 Project


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")
pd.set_option(
  "display.max_columns", 200
)  # display up to 200 columns (removes '...' hidden columns)

In [8]:
df = pd.read_csv("survey_data.csv")
df.shape

(55, 13)

In [18]:
# Drop the columns that don't agree with the Confidentiality Agreement
df = df[
  df["Agreement of Confidentiality"]
  == "I agree that the information submitted will be kept confidential, and abide by the terms of confidentiality and anonymity."
]
# Drop the Timestamp and Agreement of Confidentiality columns
df = df.drop(columns=["Timestamp", "Agreement of Confidentiality"])

df.head(10)

,"How long before you go to sleep do you usually stop using your phone, on a regular basis?",What do you do on your phone before going to sleep?,Where do you usually put your phone right before you go to sleep?,How long does it take you to fall asleep?,How often do you wake up during the night?,How many hours of sleep do you get per night on average?,"How would you rate your average sleep quality? \n(ex: waking up feeling refreshed and ready to start the day, or often feel tired and like you didn't sleep well)?","Do you have any underlying conditions that may affect the quality of your sleep? (ex: Anxiety, ADHD, etc.)",Please check all of the following that apply to your typical sleep environment:,How often do you exercise?,Any thoughts you would like to add? (Optional)
0,< 30 minutes,Watch Movies/Shows,Bedside,< 30 minutes,Rarely (1 time),6,3,No,"My room is very dark., It is usually quiet in ...",I don't exercise,NaN
1,< 30 minutes,"Social Media (Ex: Instagram, TikTok...)",Bedside,< 30 minutes,Never (0 times),6,3,Yes,"My room has some light (e.g., from a nightligh...",Less than once per week,NaN
2,< 30 minutes,"Social Media (Ex: Instagram, TikTok...)",Bedside,< 30 minutes,Never (0 times),8,4,No,"My room is very dark., There is some noise in ...",1-3 times per week,NaN
3,< 30 minutes,"Social Media (Ex: Instagram, TikTok...)",Bedside,< 30 minutes,Never (0 times),8,3,Yes,"My room has some light (e.g., from a nightligh...",1-3 times per week,NaN
4,30 minutes - 1 hour,"Social Media (Ex: Instagram, TikTok...)",In bedroom,< 30 minutes,Rarely (1 time),6,3,No,"My room is very dark., It is usually quiet in ...",1-3 times per week,slay!
5,< 30 minutes,"Social Media (Ex: Instagram, TikTok...)",Bedside,30 minutes - 1 hour,Rarely (1 time),7,2,Yes,"My room has some light (e.g., from a nightligh...",Less than once per week,NaN
6,< 30 minutes,"Social Media (Ex: Instagram, TikTok...)",Bedside,< 30 minutes,Rarely (1 time),8,3,No,"My room is very dark., It is usually quiet in ...",More than 3 times per week,NaN
7,< 30 minutes,Play games,Bedside,< 30 minutes,Rarely (1 time),4,5,Yes,"My room is very dark., It is usually quiet in ...",1-3 times per week,NaN
8,< 30 minutes,"Social Media (Ex: Instagram, TikTok...)",Bedside,< 30 minutes,Never (0 times),8,4,No,"My room is very dark., It is usually quiet in ...",Less than once per week,NaN
9,> 1 hour,Play games,Bedside,30 minutes - 1 hour,Rarely (1 time),6,3,No,"My room has some light (e.g., from a nightligh...",1-3 times per week,NaN
